In [ ]:
print("hello")

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-3.5-turbo"
#MODEL = "mixtral:8x7b"
#MODEL = "llama3"

In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.llms import Ollama
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings

if MODEL.startswith("gpt"):
    model = ChatOpenAI(api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings()
model.invoke("tell us a joke")

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain. invoke("Tell me a joke")

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("testdoc.pdf")
pages = loader.load_and_split()
pages

In [ ]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't
answer the question, reply "I don't know"."

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt. format(context="Here is some context", question="Here is a question"))

In [ ]:
chain = prompt | model | parser

In [ ]:
chain.input_schema.schema()

In [ ]:
chain.invoke(
    {
        "context": "The name I was given was Jerry",
        "question": "What's my name?"   
    }
)

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages,
    embedding = embeddings
)

In [ ]:
retriever = vectorstore.as_retriever()

retriever.invoke("cost alerts")

In [ ]:
from operator import itemgetter

chain = (
{
    "context": itemgetter("question") | retriever,
    "question": itemgetter("question")}
    | prompt
    | model
    | parser
)

chain.invoke({"question": "how to prepare and distribute audit report?"})